### Libraries

In [ ]:
#pip install cdlib
#%pip install gseapy
import pickle
import networkx as nx
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import powerlaw
from itertools import combinations
from networkx.algorithms import bipartite
from networkx.algorithms.cuts import conductance
import community.community_louvain as community_louvain
warnings.filterwarnings('ignore')
from cdlib import algorithms, evaluation
import time
import gseapy as gp
from collections import defaultdict
import seaborn as sns

### Dataset

In [ ]:
df = pd.read_csv('/Users/chiaracap/Downloads/final_dataset_ridotto.csv')

In [ ]:
df.head()

### Bipartite graph and projection on the genes

In [ ]:
# bipartite graph
B = nx.Graph()

# nodes (genes or diseases)
B.add_nodes_from(df["gene"], bipartite="gene")
B.add_nodes_from(df["disease"], bipartite="disease")

# edges gene-disease
edges = list(df.itertuples(index=False, name=None)) 
B.add_edges_from(edges)

print(f"Number of nodes in the bipartite graph: {B.number_of_nodes()}")
print(f"Number of edges in the bipartite graph: {B.number_of_edges()}")

# projection on the genes
genes = df["gene"].unique()
G_genes = bipartite.projected_graph(B, genes)

print(f"Nodes in the network of the genes: {G_genes.number_of_nodes()}")
print(f"Edges in the network of the genes: {G_genes.number_of_edges()}")

### Graph characteristics

In [ ]:
# density of the graph
# Formula: density = 2E / [N(N-1)]
density = nx.density(G_genes)

# info on the degree
degrees = [deg for _, deg in G_genes.degree()]
avg_degree = np.mean(degrees)
max_degree = np.max(degrees)
min_degree = np.min(degrees)


degree_counts = np.bincount(degrees)
nonzero_degrees = np.nonzero(degree_counts)[0]
freqs = degree_counts[nonzero_degrees]

# print results
print(f"Densità: {density:.5f}")
print(f"Grado medio: {avg_degree:.2f}")
print(f"Grado minimo: {min_degree}")
print(f"Grado massimo: {max_degree}")

# histogram of the distribution of the degrees
plt.figure(figsize=(8,5))
plt.hist(degrees, bins=30, edgecolor='black')
plt.xlabel("Degree (number of connections per gene)")
plt.ylabel("Frequency")
plt.title("Distribution of the degrees in the gene network")
plt.show()


### Implementation of community detection algorithms and comparison of the results

**Label propagation and Infomap**

In [ ]:
# Functions to compute the metrics
def compute_coverage(G, communities):
    internal_edges = 0
    for comm in communities.communities:
        internal_edges += G.subgraph(comm).number_of_edges()
    return internal_edges / G.number_of_edges()

def average_internal_density(G, communities):
    densities = []
    for comm in communities.communities:
        sub = G.subgraph(comm)
        n = sub.number_of_nodes()
        if n > 1:
            densities.append(nx.density(sub))
    return sum(densities) / len(densities) if densities else 0

def evaluate_algorithm(G, communities, algo_name, runtime):
    modularity = evaluation.newman_girvan_modularity(G, communities).score
    coverage = compute_coverage(G, communities)
    avg_density = average_internal_density(G, communities)
    return {
        "Algorithm": algo_name,
        "Communities": len(communities.communities),
        "Modularity": modularity,
        "Coverage": coverage,
        "Avg internal density": avg_density,
        "Runtime (s)": round(runtime, 2)
    }


# Dictionary to save the results

results_list = []


# LABEL PROPAGATION

start = time.time()
lp_coms = algorithms.label_propagation(G_genes)
elapsed = time.time() - start
results_list.append(evaluate_algorithm(G_genes, lp_coms, "Label Propagation", elapsed))
print(f"Label Propagation completed in {elapsed:.2f}s, {len(lp_coms.communities)} communities")


# INFOMAP

start = time.time()
infomap_coms = algorithms.infomap(G_genes)
elapsed = time.time() - start
results_list.append(evaluate_algorithm(G_genes, infomap_coms, "Infomap", elapsed))
print(f"Infomap completed in {elapsed:.2f}s, {len(infomap_coms.communities)} communities")

results_df = pd.DataFrame(results_list)
display(results_df)



**Louvain weighted**

In [ ]:
# Weighted projection on the genes
genes = df['gene'].unique()
G_genes_weighted = bipartite.weighted_projected_graph(B, genes)


In [ ]:
# Community detection and metrics computation

# Weighted Louvain
start = time.time()
louvain_coms = algorithms.louvain(G_genes_weighted, weight='weight')
elapsed = time.time() - start
print(f"Louvain weighted completed in {elapsed:.2f}s, {len(louvain_coms.communities)} communities")


# Metrics

def compute_coverage(G, communities):
    internal_edges = sum(G.subgraph(comm).number_of_edges() for comm in communities.communities)
    return internal_edges / G.number_of_edges()

def average_internal_density(G, communities):
    densities = [nx.density(G.subgraph(comm)) for comm in communities.communities if len(comm) > 1]
    return sum(densities) / len(densities) if densities else 0

modularity = evaluation.newman_girvan_modularity(G_genes_weighted, louvain_coms).score
coverage = compute_coverage(G_genes_weighted, louvain_coms)
avg_density = average_internal_density(G_genes_weighted, louvain_coms)

print("\n Metrics")
print(f"Modularity: {modularity:.4f}")
print(f"Coverage:   {coverage:.4f}")
print(f"Avg. internal density: {avg_density:.4f}")


# Avg Edge weight for each community

print("\n Avg. edge weight for each community")
community_weights = []
for i, comm in enumerate(louvain_coms.communities, 1):
    subgraph = G_genes_weighted.subgraph(comm)
    num_nodes = subgraph.number_of_nodes()
    num_edges = subgraph.number_of_edges()
    avg_weight = sum(d['weight'] for u, v, d in subgraph.edges(data=True)) / max(1, num_edges)
    print(f"Community {i}: {num_nodes} genes, {num_edges} edges, average weight {avg_weight:.2f}")
    community_weights.append(avg_weight)


# SUMMARY TABLE
results_df = pd.DataFrame([{
    "Algorithm": "Louvain (weighted)",
    "Communities": len(louvain_coms.communities),
    "Modularity": modularity,
    "Coverage": coverage,
    "Avg internal density": avg_density,
    "Avg edge weight (mean over communities)": sum(community_weights)/len(community_weights),
    "Runtime (s)": round(elapsed, 2)
}])
display(results_df)


**Conductance and Internal density on weighted Louvain**

In [ ]:
community_metrics = []

for i, comm in enumerate(louvain_coms.communities, 1):
    subG = G_genes_weighted.subgraph(comm)
    n = subG.number_of_nodes()
    m = subG.number_of_edges()
    
# conductance
    try:
        phi = conductance(G_genes_weighted, comm, weight='weight')
    except ZeroDivisionError:
        phi = 0.0

# internal density
    density = nx.density(subG) if n > 1 else 0

# average degree
    avg_degree = (2 * m / n) if n > 0 else 0

    community_metrics.append({
        "Community": i,
        "Num_nodes": n,
        "Num_edges": m,
        "Conductance": round(phi, 4),
        "Internal_density": round(density, 4),
        "Average_degree": round(avg_degree, 4)
    })

# results
metrics_df = pd.DataFrame(community_metrics)
display(metrics_df)


### Enrichment analysis

In [ ]:
# Group the genes for each Louvain weighted community

communities = defaultdict(list)
for i, comm in enumerate(louvain_coms.communities, 1):
    communities[i] = comm  


# List of databases used for enrichment

gene_sets_list = ['KEGG_2021_Human', 'GO_Biological_Process_2021', 'DisGeNET']


# Function used for enrichment

def enrich_community(gene_list, gene_sets=gene_sets_list, top_n=5):
    results_dict = {}
    for gs in gene_sets:
        enr = gp.enrichr(
            gene_list=gene_list,
            gene_sets=[gs],
            organism='Human',
            outdir=None  # non salva su disco
        )
        # top n most significative terms
        top_terms = enr.results[['Term','Overlap','Adjusted P-value']].sort_values('Adjusted P-value').head(top_n)
        results_dict[gs] = top_terms
    return results_dict


# Cycle on all the communities

all_enrichments = {}
for comm_id, genes in communities.items():
    if len(genes) < 3:  # we exclude the communities with less than 2 genes
        continue
    all_enrichments[comm_id] = enrich_community(genes, gene_sets_list, top_n=5)

# Table 

for comm_id, enrich_dict in all_enrichments.items():
    print(f"\n=== Community {comm_id} ===")
    for gs_name, df in enrich_dict.items():
        print(f"\nDatabase: {gs_name}")
        print(df.to_string(index=False))

# Visualization

fig, axes = plt.subplots(len(all_enrichments), 1, figsize=(8, len(all_enrichments)*2.5))
if len(all_enrichments) == 1:
    axes = [axes]

for ax, (comm_id, enrich_dict) in zip(axes, all_enrichments.items()):
    df_kegg = enrich_dict['KEGG_2021_Human']
    ax.barh(df_kegg['Term'], -np.log10(df_kegg['Adjusted P-value']))
    ax.set_title(f'Community {comm_id} - KEGG enrichment')
    ax.set_xlabel('-log10(adj p-value)')
    ax.invert_yaxis()  

plt.tight_layout()
plt.show()


In [ ]:
# GROUP GENED BY WEIGHTED LOUVAIN
communities = defaultdict(list)
for i, comm in enumerate(louvain_coms.communities, 1):
    communities[i] = comm  # usa indici numerici da 1 in poi


# Database of interest

gene_sets_list = ['KEGG_2021_Human', 'GO_Biological_Process_2021', 'DisGeNET']


# Function to do the enrichment

def enrich_community(gene_list, gene_sets=gene_sets_list):
    enr = gp.enrichr(
        gene_list=gene_list,
        gene_sets=gene_sets,
        organism='Human',
        outdir=None
    )
    return enr.results


# Cycle on all the communities

all_results = []
for comm_id, genes in communities.items():
    if len(genes) < 3:  # evita enrichment su community troppo piccole
        continue
    res = enrich_community(genes, gene_sets_list)
    res["Community"] = comm_id
    all_results.append(res)

df_all = pd.concat(all_results, ignore_index=True)


# Filter only the significative terms

df_sig = df_all[df_all["Adjusted P-value"] < 0.05].copy()
print(f"Total of the found significative terms: {len(df_sig)}")


# Construct the term x community matrix

df_sig["score"] = -np.log10(df_sig["Adjusted P-value"])

heatmap_data = df_sig.pivot_table(
    index="Community",
    columns="Term",
    values="score",
    aggfunc="max",
    fill_value=0
)


# Heatmap

plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data, cmap="YlGnBu", cbar_kws={'label': '-log10(adj p-value)'})
plt.title("Louvain weighted communities × Disease/Pathway (significant terms)")
plt.ylabel("Community")
plt.xlabel("Pathway / Disease")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


# Top 5 terms for each community

top_terms = (
    df_sig.sort_values(by=["Community", "score"], ascending=[True, False])
    .groupby("Community")
    .head(5)
)

top_terms_table = top_terms[["Community", "Term", "Adjusted P-value", "score"]]
top_terms_table = top_terms_table.sort_values(["Community", "score"], ascending=[True, False])

# table
print("\nSummary table with the top 5 terms for each community")
display(top_terms_table)
